<h3> ChromaDB Crash Course</h3>

<p style="font-size:16px"> ChromaDB is a vector database that automatically computes and stores embeddings for text data. When a query is made, it compares the embeddings using distance metrics and returns the most relevant results.</p>

In [4]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


Create a DB


In [5]:
import chromadb 
chroma_client = chromadb.Client()

create collection

In [6]:
collection = chroma_client.create_collection(name="My_Collection")

Add some text document to collection

In [7]:
collection.add(
    documents=[
        "This is a document about  ",
        "Welcome to this documente"
    ],
    ids=['id1', 'id2']
)

Query the collection

In [8]:
results=collection.query(
    query_texts=["This is a sample query about hawaii"], # chroma will embed this 
    n_results=2 # No of result to return
)

print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about  ', 'Welcome to this documente']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[1.5123367309570312, 1.9398043155670166]]}


In [9]:
results=collection.query(
    query_texts=["This is a sample query about Orange"], # chroma will embed this 
    n_results=2 # No of result to return
)

print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about  ', 'Welcome to this documente']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[1.6396170854568481, 1.9013142585754395]]}


Persist the data

In [10]:
client = chromadb.PersistentClient(path="./db/")

In [11]:
client.heartbeat()  # returns a nanosec heartbeat to make sure the client remain connected

1747466652898853800

In [12]:
client.reset() # By simply runnin this will not work

AuthorizationError: Reset is disabled by config

In [13]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [14]:
DEFAULT_TENANT,DEFAULT_DATABASE

('default_tenant', 'default_database')

In [15]:
client = chromadb.PersistentClient(
    path="./db2/",
    settings=Settings(
                is_persistent = True,
                persist_directory = "/db2/",
                allow_reset = True,
        anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# path - parameter must be a local path on the machine where Chroma is running. If the path does not exist, it will be created. The path can be relative or absolute. If the path is not specified, the default is ./chroma in the current working directory.
# settings - Chroma settings object.
# tenant - the tenant to use. Default is default_tenant.
# database - the database to use. Default is default_database.

In [16]:
client.reset()  # Now this will work

True

<b> Creating, Inspecting and Deleting Collection </b>

<p>
Chroma uses collection names in the url, so there are a few restrictions on naming them:

- The length of the name must be between 3 and 63 characters.
- The name must start and end with a lowercase letter or a digit, and it can contain dots, dashes, and underscores in between.
- The name must not contain two consecutive dots.
- The name must not be a valid IP address.</p>

<italics>Chroma collections are created with a name and an optional embedding function. If you supply an embedding function, you must supply it every time you get the collection.</italics >

In [17]:
from chromadb.utils import embedding_functions

In [18]:
emb_fun=embedding_functions.SentenceTransformerEmbeddingFunction()

d:\# My Learning\Coding\Summer-Internship\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model_name = "all-MiniLM-L6-v2"
emb_fun = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

In [20]:
emb_fun(["Welcome"])

[array([-5.73623739e-02, -1.16481781e-02, -1.22139952e-03,  2.09228769e-02,
         2.38797851e-02, -3.77398245e-02, -2.04281565e-02, -2.06078868e-02,
        -4.46448885e-02, -2.95937173e-02,  3.68522406e-02,  5.80347553e-02,
        -6.66918010e-02,  1.99460499e-02, -6.76470697e-02,  6.63192570e-02,
         7.03084841e-02, -1.20073380e-02, -2.81319190e-02, -5.41426837e-02,
         4.89913998e-03, -4.27241176e-02,  5.71595831e-03,  3.22622769e-02,
        -4.49759774e-02, -1.69816334e-02,  3.40951197e-02,  6.09736741e-02,
         1.69017669e-02, -3.45815085e-02, -4.21386994e-02,  7.32292309e-02,
         4.14056964e-02,  9.79096349e-03,  2.84802783e-02, -2.45167073e-02,
         2.18786951e-02, -2.08951533e-02, -3.16071697e-02, -1.68134626e-02,
         1.00759100e-02, -2.11724937e-02, -4.47666273e-02,  1.13326788e-03,
        -5.24000973e-02,  1.01943985e-01, -4.18204255e-02, -4.08316217e-02,
         1.66139044e-02,  5.19749448e-02,  4.90134489e-03, -5.97857637e-03,
        -4.8

In [21]:
len(emb_fun(["Welcome"])[0])

384

Create Collection

In [22]:
collection=client.create_collection(name="My_Collection2", embedding_function=emb_fun)

Get collection

In [23]:
collection=client.get_collection(name="My_Collection2", embedding_function=emb_fun)

Delete Collection

In [24]:
client.delete_collection(name="My_Collection2")

IF Not sure if a collection exist or need to be created

In [25]:
collection=client.get_or_create_collection(name="Some_Collection")

In [26]:
collection

Collection(name=Some_Collection)

Rename a collection


In [27]:
collection.modify(name="New_Collection_Name")
collection

Collection(name=New_Collection_Name)

Add Documents

In [28]:
collection.add(
    documents=["som2", "doc2", "doc3"],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)

<p style="Font-size:14px">If Chroma is passed a list of documents, it will automatically tokenize and embed them with the collection's embedding function (the default will be used if none was supplied at collection creation). Chroma will also store the documents themselves. If the documents are too large to embed using the chosen embedding function, an exception will be raised. </p>
<p style="Font-size:14px">
Each document must have a unique associated id. Trying to .add the same ID twice will result in only the initial value being stored. An optional list of metadata dictionaries can be supplied for each document, to store additional information and enable filtering.<br></p>
<p style="Font-size:14px">
Alternatively, you can supply a list of document-associated embeddings directly, and Chroma will store the associated documents without embedding them itself.</p>

In [29]:
collection.add(
    documents=["doc1", "doc2", "doc3"],
#     embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id11", "id21", "id31"]
)

In [30]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [31]:
doc_lists=[]
metda_list=[]
for i in newsgroups_train['data'][:100]:
    doc_lists.append(i)
    metda_list.append({'len_of_doc':len(i)})

In [33]:
collection.add(
    documents=doc_lists,
#     embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=metda_list,
    ids=['id_{}'.format(i) for i in range(100)]
)

select 10 records

In [34]:
collection.peek(2) # returns a list of the first 10 items in the collection

{'ids': ['id1', 'id2'],
 'embeddings': array([[-7.27678509e-03, -3.58610414e-02, -5.85470051e-02,
          4.30398919e-02,  2.56231036e-02, -2.53969394e-02,
          3.37159522e-02,  3.43396189e-03, -2.63475925e-02,
         -4.61397991e-02,  8.30679685e-02, -6.42239768e-03,
         -3.55973691e-02, -3.96921635e-02,  4.66601700e-02,
          2.61252634e-02,  8.53319243e-02, -1.68907885e-02,
         -1.70462276e-03, -1.81527901e-02,  6.35295585e-02,
         -3.71037200e-02,  5.74298874e-02,  2.37502102e-02,
         -4.89793792e-02,  3.40397581e-02,  4.02286369e-03,
          6.53342456e-02,  6.25966191e-02, -1.67348072e-01,
          7.33717456e-02,  9.65449214e-02, -2.70988718e-02,
          6.61829999e-03, -8.19215104e-02, -2.82600895e-02,
          3.99508812e-02, -9.23850313e-02, -9.49947312e-02,
         -1.11495787e-02,  3.19334329e-03,  3.37751233e-03,
         -1.56492591e-02, -4.09619361e-02,  3.89017239e-02,
          1.65557954e-02,  4.70884219e-02, -7.38915289e-03,
  

Get number of documents in a collection

In [35]:
collection.count() # returns the number of items in the collection

106

In [37]:
collection

Collection(name=New_Collection_Name)

Define a alternate distance function

In [38]:
collection = client.create_collection(
        name="collection_name",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

Query the collections

In [41]:
collection.query(
    query_embeddings=[[11.1, 12.1, 13.1], [1.1, 2.3, 3.2]],
    n_results=10,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains": "search_string"}
)

{'ids': [[], []],
 'embeddings': None,
 'documents': [[], []],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[], []],
 'distances': [[], []]}

et also supports the where and where_document filters. If no ids are supplied, it will return all items in the collection that match the where and where_document filters.

Choosing which data is returned#

When using get or query you can use the include parameter to specify which data you want returned - any of embeddings, documents, metadatas, and for query, distances. By default, Chroma will return the documents, metadatas and in the case of query, the distances of the results. embeddings are excluded by default for performance and the ids are always returned. You can specify which of these you want returned by passing an array of included field names to the includes parameter of the query or get method.

In [42]:
collection.get(
    include=["documents"]
)

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['documents'],
 'data': None,
 'metadatas': None}

In [44]:
collection.query(
    query_embeddings=[[11.1, 12.1, 13.1], [1.1, 2.3, 3.2]],
    include=["documents"]
)

{'ids': [[], []],
 'embeddings': None,
 'documents': [[], []],
 'uris': None,
 'included': ['documents'],
 'data': None,
 'metadatas': None,
 'distances': None}

Using Where filters#

Chroma supports filtering queries by metadata and document contents. The where filter is used to filter by metadata, and the where_document filter is used to filter by document contents.

Filtering by metadata#

In order to filter on metadata, you must supply a where filter dictionary to the query. The dictionary must have the following structure:

{ "metadata_field": { : } }

Filtering metadata supports the following operators:
* $eq - equal to (string, int, float)

$ne - not equal to (string, int, float)

$gt - greater than (int, float)

$gte - greater than or equal to (int, float)

$lt - less than (int, float)

$lte - less than or equal to (int, float)

Filtering for a search_string

{ "$contains": "search_string" }

Using logical operators#

You can also use the logical operators 
or to combine multiple filters.

An $and operator will return results that match all of the filters in the list.

{ "$and": [ { "metadata_field": { : } }, { "metadata_field": { : } } ] }

{ "$or": [ { "metadata_field": { : } }, { "metadata_field": { : } } ] }

Update

In [ ]:
collection.update(
    ids=['id_{}'.format(i) for i in range(100)],
    embeddings=emb_fun(doc_lists),
    metadatas=metda_list,
    documents=doc_lists,
)

Find and Update if not found add

In [ ]:
collection.upsert(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)

Deleting data from a collection#

Chroma supports deleting items from a collection by id using .delete. The embeddings, documents, and metadata associated with each item will be deleted. ⚠️ Naturally, this is a destructive operation, and cannot be undone.

In [ ]:
collection.delete(
    ids=["id1", "id2", "id3",...],
	where={"chapter": "20"}
)